# Mutant Generation

### Prepare Masculine and Feminine Word

In [7]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv("asset/masculine-feminine.txt")
df.head()

,masculine,feminine
0,actor,actress
1,author,authoress
2,boy,girl
3,bridegroom,bride
4,brother,sister


In [9]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from string import punctuation
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

def list_to_string(l):
    return " ".join(l)

wordnet_lemmatizer = WordNetLemmatizer()
snowball_stemmer = SnowballStemmer("english")

text = "This is a Demo Text for NLP using NLTK. Full form of NLTK is Natural Language Toolkit"

def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

def lemmatize_stemming(text) :
    text = text.lower()
    text = strip_punctuation(text)
    word_tokens = nltk.word_tokenize(text)
    word_tokens = [wordnet_lemmatizer.lemmatize(word) for word in word_tokens]
    word_tokens = [snowball_stemmer.stem(word) for word in word_tokens]
    return list_to_string(word_tokens)
    
def lowercasing(text):
    return text.lower()

lemmatize_stemming(text)

'this is a demo text for nlp use nltk full form of nltk is natur languag toolkit'

In [11]:
df["masculine"] = df["masculine"].apply(lowercasing)
df["feminine"] = df["feminine"].apply(lowercasing)

In [12]:
df = df.drop_duplicates().reset_index(drop=True)

In [13]:
df.to_csv("asset/masculine-feminine-cleaned.txt", index=False)

### Mutant Generation

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [32]:
replacement = pd.read_csv("asset/masculine-feminine-cleaned.txt")

m = {}
mlist = []
f = {}
flist = []
for index, row in replacement.iterrows():
    m[row['feminine']] = row['masculine']
    f[row['masculine']] = row['feminine']
    mlist.append(row["masculine"])
    flist.append(row["feminine"])

In [33]:
mnames = ["James", "John", "Robert", "Michael", "Whilliam", "Richard"]
fnames = ["Jessica", "Patricia", "Jennifer", "Linda", "Elizabeth"]

# masculine pronoun
mp = ["He", "He's", "he", "he's", "him", "his", "himself"]

# feminine prononun
fp = ["She", "He's", "she", "he's","her", "her", "herself"]

mc = {}
fc = {}

for _m, _f in zip(mp, fp) :
    fc[_m] = _f
    mc[_f] = _m

In [34]:
text = "gingerAZ. Bronson's character plans to live anonymously but when he finds his fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Bronson's character"
# text = "Good songs....but good album? debateable. She has some rockin songs on here. All her singles kickass and the last song on the album with M.I.A. (i friggin love her) is fire. but then, i start listening to the other songs, ready for a song on the album that is just as good as her singles and i can't find another. All of missy's singles are crazy, but then ui listen to the album, and its like a let down. any song that she sings in makes my stomach churn and some have just annoying beats and repititious phrases. If you really liked Missy's album 'This Is Not A Test', then get this album. but if you are lovin her singles and want some more, just download the singles, don't waste your money on the rest of this album"
# text = "A landlord book that's not based on today's reality.. Although Shemin brings up many good ideas, I found that most of his ideas are way to cheezy for me (or Southern california.) He discusses buying places for $8,000-$20,000 (I wish) and believes that charging more for rent will ultimately get you higher quality tenents. I completely disagree, especially in down markets.A decent book, overall, but I'd suggest a Nolo book which has more real legal information as well as solid do's and don'ts for a landlord."
# text = "He discusses buying places for $8,000-$20,000 (I wish) and believes that charging more for rent will ultimately get you higher quality tenents."
# text = ""

In [35]:
# nltk.download('averaged_perceptron_tagger')

In [36]:
def pos_tagging(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

def get_pronoun(text) :
    
    pt = pos_tagging(text)
    
    # subject pronoun
    spr = set()
    
    #non-subject prononun
    pr = set()

    for (word, tag) in pt :
        if "PRP" == tag :
            spr.add(word)
        elif "PRP$" == tag :
            pr.add(word)
    
    return spr, pr

spr, npr = get_pronoun(text)
print(spr)
print(npr)

{'he'}
{'his'}


In [37]:
def get_person(text) :
    doc = nlp(text)

    person = set()

    for ent in doc.ents:
#         print(ent.text, ent.start_char, ent.end_char, ent.label_)
        if (ent.label_ == "PERSON") :
            person.add(ent.text)
    
    return person
    
person = get_person(text)
person

{'Bronson'}

In [53]:
MALE = "m"
FEMALE = "f"
UNIDENTIFIED = "u"

def get_gender(text) :
    spr, npr = get_pronoun(text)
    male = False
    female = False
    if "he" in spr or "He" in spr :
        male = True
    if "she" in spr or "She" in spr :
        female = True
    
    if male != female :
        if male :
            return MALE
        else :
            return FEMALE
    else :
        return UNIDENTIFIED

get_gender(text)

'u'

In [39]:
import tokenizer

def replace_pronoun(text) :
    
    is_replaced = False

    toklist = list(tokenizer.tokenize(text))

    # print(toklist)
    newtoklist = []
    for token in toklist :
        if token.txt in mp :
            token = token._replace(txt = fc[token.txt])
            newtoklist.append(token)
            is_replaced = True
        elif token.txt in fp :
            token = token._replace(txt = mc[token.txt])
            newtoklist.append(token)
            is_replaced = True
        else :
            newtoklist.append(token)

    # print(newtoklist)

    return is_replaced, tokenizer.detokenize(newtoklist, normalize=True)

print(text)
_, _text = replace_pronoun(text)
print(_text)

gingerAZ. Bronson's character plans to live anonymously but when he finds his fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Bronson's character
gingerAZ. Bronson's character plans to live anonymously but when she finds her fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Bronson's character


In [40]:
def is_set_empty(s) :
    return not bool(s)

def generate_mutant_from_text(text) :

    mutant = []

    is_replaced, text_with_replaced_pronoun = replace_pronoun(text)

    if is_replaced :

        person = get_person(text)

        if is_set_empty(person) :
            mutant.append(text_with_replaced_pronoun)
    #         print("No name")
        elif len(person) == 1 :
    #         print("Found 1 name")

            spr, npr = get_pronoun(text)
            gender = get_gender(spr)

            if gender == MALE :

                # get the name
                prevname = person.pop()

                for name in fnames :    
                    # replace the name
                    temp_text = text_with_replaced_pronoun.replace(prevname, name)
                    mutant.append(temp_text)

            elif gender == FEMALE :

                # get the name
                prevname = person.pop()

                for name in mnames :    
                    # replace the name
                    temp_text = text_with_replaced_pronoun.replace(prevname, name)
                    mutant.append(temp_text)

            else :
                mutant.append(text_with_replaced_pronoun)

    return mutant

print(text)
generate_mutant_from_text(text)

gingerAZ. Bronson's character plans to live anonymously but when he finds his fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Bronson's character


["gingerAZ. Jessica's character plans to live anonymously but when she finds her fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Jessica's character",
 "gingerAZ. Patricia's character plans to live anonymously but when she finds her fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Patricia's character",
 "gingerAZ. Jennifer's character plans to live anonymously but when she finds her fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Jennifer's character",
 "gingerAZ. Linda's character plans to live anonymously but when she finds her fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Linda's character",
 "gingerAZ. Elizabeth's character plans to live anonymously but when she finds her fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Eliza

### Generate Mutant from IMDB Data

In [57]:
df = pd.read_csv("imdb/test.csv", sep="\t", header=None, names=["label", "text"])
df = df[:100]
df

,label,text
0,1,The Mascot is Ladislaw Starewicz's masterpiece...
1,1,this is one of the finest movies i have ever s...
2,1,"I have only see three episodes of Hack, starri..."
3,1,"Deepa Mehta's ""Fire"" is groundbreaking, bold, ..."
4,0,Anyone who sees this film will notice that the...
...,...,...
95,0,I'm sorry but I can't agree that this was a go...
96,1,"When I was younger, this movie always aired on..."
97,0,I love basketball and this seemed like an intr...
98,1,I've been looking forward to seeing this film ...


In [58]:
df["gender"] = df["text"].apply(get_gender)

In [59]:
df = df[df["gender"] != UNIDENTIFIED]
df = df.reset_index(drop=True)
df

,label,text,gender
0,1,"I have only see three episodes of Hack, starri...",m
1,1,In the groovy mid 70's a scruffy bunch of bras...,m
2,0,The sequel that no one asked for to the movie ...,m
3,0,"As many know, this is the feature film debut o...",m
4,1,This must have been one of Chaplin's most ambi...,m
5,1,"One of the oddest, most strikingly eerie and c...",m
6,1,I will freely admit that I haven't seen the or...,f
7,1,The debut that plucked from obscurity one of t...,m
8,1,There is really no way to compare this motion ...,m
9,1,Love is a Many-Splendored Thing is set in Hong...,f


In [113]:
text = df["text"][30]
text

'When I was younger, this movie always aired on Friday night in the summer on Channel 40 (this was the years before Fox was a network and took over the programming). I always looked forward to it. I\'d go grocery shopping with my parents, then sit down with my Swanson\'s TV dinner and a Lady Lee Cola(the only time of the week I was allowed to drink cola, and enjoy. Sure, the script is predictably late 70\'s (like Little Darlings), but it\'s a fun movie, and I loved Rudy and Tripper. Bill Murray coasts with little effort in the movie, but he is charming. Gotta love Spaz and those taped glasses (pre Revenge of the Nerds). Chris Makepeace is pretty much the same character he played in "My Bodyguard" but he does it so well.'

In [114]:
get_gender(text)

'm'

### Coreference Task

In [14]:
# !python -m spacy download en_core_web_sm
# !python -m spacy download xx_ent_wiki_sm
# !python -m spacy download en_core_web_lg

In [15]:
import spacy
# import en_core_web_sm
# nlp = en_core_web_sm.load()
# import xx_ent_wiki_sm
# nlp = xx_ent_wiki_sm.load()
import en_core_web_lg
nlp = en_core_web_lg.load()

# !pip install neuralcoref
import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

In [27]:
# You're done. You can now use NeuralCoref the same way you usually manipulate a SpaCy document and it's annotations.
doc = nlp("My sister has a dog. She loves him. She likes him")

doc._.has_coref
doc._.coref_clusters

[My sister: [My sister, She, She], a dog: [a dog, him, him]]

In [28]:
doc._.coref_clusters[0].mentions

[My sister, She, She]

In [201]:
doc = nlp(text)

doc._.has_coref
doc._.coref_clusters[0].main

this movie

In [191]:
class Ref:
    
    name = ""
    reference = []
    
    def __init__(self, name, reference):
        self.name = str(name)
        self.reference = []
        for word in reference :
            self.reference.append(str(word))
            
    def __str__(self) :
        return self.name + ": " + str(self.reference)
    
    def get_name(self):
        return self.name
    
    def get_reference(self):
        return self.reference
    
    def is_male(self):
        if "He" in self.reference :
            return True
        elif "he" in self.reference :
            return True
        else :
            return False

    def is_female(self):
        if "She" in self.reference :
            return True
        elif "she" in self.reference :
            return True
        else :
            return False

In [192]:
doc._.coref_clusters[2].mentions[1]

he

In [193]:
doc._.coref_clusters[2].mentions[0]

Bill Murray

In [194]:
r = Ref(doc._.coref_clusters[2].main, doc._.coref_clusters[2].mentions)

In [195]:
r.is_male()

True

In [203]:
class Coref:
    original = ""
    resolved = ""
    refs = []
    
    def __init__(self, original, resolved, refs):
        self.original = str(original)
        self.resolved = str(resolved)
        for r in refs :
            print(r.main)
            print(r.mentions)
            self.refs.append(Ref(r.main, r.mentions))
            
    def get_resolved(self):
        return self.resolved
    
    def get_refs(self):
        return self.refs

In [204]:
c = Coref(text, doc._.coref_resolved, doc._.coref_clusters)

this movie
[this movie, the movie]
the script
[the script, it]
Bill Murray
[Bill Murray, he, he, he]


In [205]:
c.get_resolved()

'When I was younger, this movie always aired on Friday night in the summer on Channel 40 (this was the years before Fox was a network and took over the programming). I always looked forward to it. I\'d go grocery shopping with my parents, then sit down with my Swanson\'s TV dinner and a Lady Lee Cola(the only time of the week I was allowed to drink cola, and enjoy. Sure, the script is predictably late 70\'s (like Little Darlings), but the script\'s a fun movie, and I loved Rudy and Tripper. Bill Murray coasts with little effort in this movie, but Bill Murray is charming. Gotta love Spaz and those taped glasses (pre Revenge of the Nerds). Chris Makepeace is pretty much the same character Bill Murray played in "My Bodyguard" but Bill Murray does it so well.'

In [206]:
for r in c.get_refs() :
    print(r.get_name())
    print(r.get_reference())

this movie
['this movie', 'the movie']
the script
['the script', 'it']
Bill Murray
['Bill Murray', 'he', 'he', 'he']


In [108]:
doc = nlp('My sister has a dog. She loves him')
token = doc[-1]
span = doc[-1:]


print(doc._.coref_resolved)
print(doc._.coref_clusters)
print(doc._.coref_clusters[1].mentions)
print(doc._.coref_clusters[1].mentions[-1])
print(doc._.coref_clusters[1].mentions[-1]._.coref_cluster.main)
print(token._.in_coref)
print(token._.coref_clusters)
print(span._.is_coref)
print(span._.coref_cluster.main)
print(span._.coref_cluster.main._.coref_cluster)

My sister has a dog. My sister loves a dog
[My sister: [My sister, She], a dog: [a dog, him]]
[a dog, him]
him
a dog
True
[a dog: [a dog, him]]
True
a dog
a dog: [a dog, him]
